<a href="https://colab.research.google.com/github/MattyXarope/Comp-Sci-Homework-3-/blob/main/Assignment_3_1_2_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [148]:
import re
import nltk
import base64
import requests
import pandas as pd
from pandas import *
from nltk.tag.perceptron import PerceptronTagger
from google.colab import drive
drive.mount('/content/drive')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Implement Hearst Pattern Function

In [149]:
class HearstPatterns(object):

    def __init__(self, extended=False):
        self.__chunk_patterns = r""" #  helps us find noun phrase chunks
                NP: {<DT>?<JJ.*>*<NN.*>+}
                    {<NN.*>+}
                """
        # create a chunk parser
        self.__np_chunker = nltk.RegexpParser(self.__chunk_patterns)

        # now define the Hearst patterns
        # format is <hearst-pattern>, <hypernym_location>
        # so, what this means is that if you apply the first pattern,
        self.__hearst_patterns=[ (
                '(NP_\\w+ (, )?such as (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(such NP_\\w+ (, )?as (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?other NP_\\w+)',
                'last'
            ),
            (
                '(NP_\\w+ (, )?include (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?includes (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?including (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?especially (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                    '((NP_\\w+ ?(, )?)+(and |or )?any other NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?some other NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?be a NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?like (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    'such (NP_\\w+ (, )?as (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?like other NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?one of the NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?one of these NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?one of those NP_\\w+)',
                    'last'
                ),
                (
                    'example of (NP_\\w+ (, )?be (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?be example of NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?for example (, )?'
                    '(NP_\\w+ ?(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which be call NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which be name NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?mainly (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?mostly (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?notably (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?particularly (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?principally (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?in particular (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?except (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?other than (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?e.g. (, )?(NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ \\( (e.g.|i.e.) (, )?(NP_\\w+ ? (, )?(and |or )?)+'
                    '(\\. )?\\))',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?i.e. (, )?(NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? a kind of NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? kind of NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? form of NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which look like NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which sound like NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?which be similar to (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?example of this be (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?type (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )? NP_\\w+ type)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?whether (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(compare (NP_\\w+ ?(, )?)+(and |or )?with NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?compare to (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?among -PRON- (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?as NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )? (NP_\\w+ ? (, )?(and |or )?)+ '
                    'for instance)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? sort of NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?which may include (NP_\\w+ '
                    '?(, )?(and |or )?)+)',
                    'first'
                ),
                ]

        if extended:
            self.__hearst_patterns.extend([
                ("(NP_\w+ (, )?such as (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?like (NP_\\w+ ? (, )?(and |or )?)+)",
                    'first')
            ])

        self.__pos_tagger = PerceptronTagger()

    def prepare(self, rawtext):
        # To process text in NLTK format
        sentences = nltk.sent_tokenize(rawtext.strip())
        sentences = [nltk.word_tokenize(sent) for sent in sentences]
        sentences = [self.__pos_tagger.tag(sent) for sent in sentences]

        return sentences

    def chunk(self, rawtext):
        sentences = self.prepare(rawtext.strip())

        all_chunks = []
        for sentence in sentences:
            chunks = self.__np_chunker.parse(sentence)
            all_chunks.append(self.prepare_chunks(chunks))

        # two or more NPs next to each other should be merged into a single NP,
        # find any N consecutive NP_ and merge them into one...
        # Eg: "NP_foo NP_bar blah blah" becomes "NP_foo_bar blah blah"
        all_sentences = []
        for raw_sentence in all_chunks:
            sentence = re.sub(r"(NP_\w+ NP_\w+)+",
                              lambda m: m.expand(r'\1').replace(" NP_", "_"),
                              raw_sentence)
            all_sentences.append(sentence)

        return all_sentences

    def prepare_chunks(self, chunks):
        # If chunk is NP, start with NP_ and join tokens in chunk with _
        # Else just keep the token as it is

        terms = []
        for chunk in chunks:
            label = None
            try:
                # gross hack to see if the chunk is simply a word or a NP, as
                # we want. But non-NP fail on this method call
                label = chunk.label()
            except:
                pass

            if label is None:  # means one word...
                token = chunk[0]
                terms.append(token)
            else:
                np = "NP_"+"_".join([a[0] for a in chunk])
                terms.append(np)
        return ' '.join(terms)

    def find_hyponyms(self, rawtext):

        hypo_hypernyms = []
        np_tagged_sentences = self.chunk(rawtext)

        for sentence in np_tagged_sentences:

            for (hearst_pattern, parser) in self.__hearst_patterns:
                matches = re.search(hearst_pattern, sentence)
                if matches:
                    match_str = matches.group(0)

                    nps = [a for a in match_str.split() if a.startswith("NP_")]

                    if parser == "first":
                        hypernym = nps[0]
                        hyponyms = nps[1:]
                    else:
                        hypernym = nps[-1]
                        hyponyms = nps[:-1]

                    for i in range(len(hyponyms)):
                        hypo_hypernyms.append(
                            (self.clean_hyponym_term(hyponyms[i]),
                             self.clean_hyponym_term(hypernym)))

        return hypo_hypernyms

    def clean_hyponym_term(self, term):
        return term.replace("NP_", "").replace("_", " ")

# Test Hearst Pattern Function

In [150]:
# experiment with sentences of each pattern
if __name__=='__main__':
    hp = HearstPatterns(extended=False)
    texts = [('I like to listen to music from musical genres such as blues, rock and jazz.'),
            ('I dont like foods such as mango, watermelon and pancake.'),
             'I have been to such countries as China, USA and Spain.',
             'I love animals like cat and dog.',
             "actress including Lisa, Mina and Sana.",
             "Mozart's musical compositions include symphonies and operas.",
             "I like use Electronic products, for example, Ipad and Iphone.",
             "apple is a fruit.",
             "I like dog, cat and other pets."]
    for text in texts:         
        hps = hp.find_hyponyms(text)
        print(hps)

[('blues', 'musical genres'), ('rock', 'musical genres'), ('jazz', 'musical genres')]
[('mango', 'foods'), ('watermelon', 'foods'), ('pancake', 'foods')]
[('such countries', 'China')]
[('cat', 'animals'), ('dog', 'animals')]
[('Lisa', 'actress'), ('Mina', 'actress'), ('Sana', 'actress')]
[('symphonies', 'musical compositions'), ('operas', 'musical compositions')]
[]
[]
[]


# Set Training Set for Our Data 

In [157]:
# Training Data from Gemma's Corpus - originally this was taken from the first 1000 characters but it's now just using the whole first file of the corpus.
path = "/content/drive/MyDrive/Classes/Classes 2021/Computational Semantics /Assignment 3/data/raw.en/englishText_0_10000"
file = open(path, "r", encoding='latin-1')
FileContent = file.read() 
texts=[x for x in map(str.strip, FileContent.split('.')) if x]
example = texts[:1000]

# Extract Hypernyms from Data using Hearst Pattern Function

In [158]:
#get a hypernym dictionary: hypo: ["hyper1","hyper2"....]
#improments can be done to reorden the hypernyms if exist more than one

pairs_list=[]
if __name__=='__main__':
    hp = HearstPatterns(extended=False)
    for text in texts:         
        hps = hp.find_hyponyms(text)
        if hps != []:
          for x in hps:
            pairs_list.append(x)
print(pairs_list)

hps_dict = {}
for k_v in pairs_list:
 k, v = k_v
 hps_dict.setdefault(k, []).append(v)
print(hps_dict)

[('the time', 'bears'), ('such sanctified names', 'Charles'), ('Macaulay', 'Hallam'), ('Shakespeare', 'Names'), ('Grotius', 'Names'), ('Francis Bacon', 'Names'), ('Thomas Hobbes', 'Names'), ('universities', 'institutions'), ('academies', 'institutions'), ('a philosophy', 'anything'), ('a woodworking shop', 'buildings'), ('a library', 'buildings'), ('a managed woodlot', 'facilities'), ('a challenge course', 'facilities'), ('a 30', 'facilities'), ('Jheri Redding', 'other personal health care products'), ('Rusk', 'other personal health care products'), ('consumer electronic products', 'other personal health care products'), ('telephones', 'consumer electronic products'), ('Cuisinart products', 'consumer electronic products'), ('the Guinness World Record', 'the largest securities'), ('Stamford', 'a Manufacturing Center'), ('an illustration', 'The cover'), ('Cimbalom', 'other traditional instruments'), ('Maria Montessori', 'educational pioneers'), ('Rudolf Steiner', 'educational pioneers'),

# Set Hypernyms and Hyponyms as values and Print to CSV



In [159]:
#Print the hyponyms from our tests to a csv file

#Set the keys and values as lists to call easier
hypo = hps_dict.keys()
hyper = hps_dict.values()

#Print these lists
print(hypo)
print(hyper)

#Write the hyponyms to a csv file, commented out to not do it over and over again
#hypodf = pd.DataFrame(hypo, columns=['Hyponyms'])
#hypodf.to_csv('hypo.csv')

dict_keys(['the time', 'such sanctified names', 'Macaulay', 'Shakespeare', 'Grotius', 'Francis Bacon', 'Thomas Hobbes', 'universities', 'academies', 'a philosophy', 'a woodworking shop', 'a library', 'a managed woodlot', 'a challenge course', 'a 30', 'Jheri Redding', 'Rusk', 'consumer electronic products', 'telephones', 'Cuisinart products', 'the Guinness World Record', 'Stamford', 'an illustration', 'Cimbalom', 'Maria Montessori', 'Rudolf Steiner', 'Leo Tolstoy', 'Fransisco Ferrer y Guardia', 'education', 'John Caldwell Holt', 'Paul Goodman', 'Frederick Mayer', 'Ivan Illich', 'Herbert Kohl', 'Jonathan Kozol', 'Sands School', 'Summerhill School', 'Sudbury Valley School', 'Krishnamurti schools', 'open classroom schools', 'Mountview Alternative School', 'Tarot', 'psychic ability', 'esoteric systems', 'Divine Class', 'open knowledge', 'Tom Neal', 'aster yellows', 'Catapult', 'rails', 'halfpipes', 'Alley Cat', 'a favorite intermediate cruiser', 'Catamount', 'the limit', 'Tekka Centre', 'th

# Function to get Hypernym of Input Term

In [175]:
# Function to get a list of hypernyms of the input term
def get_hps(hps_dict,term):
  return [v for k,v in hps_dict.items() if k == term]
get_hps(hps_dict, "video")



[[['enhanced content', 'Invasion', 'work']], [['practice vaulting surfaces']]]


# Imports The Task's Training Data Set and the CSV for our Hearst Training Data Set

In [155]:
#Reads the training sets and makes a list out of them. Also reads the temporary Hearst data so you don't have to run it every time. 

## Training data from English task
#Reads the csv
data = read_csv("https://raw.githubusercontent.com/MattyXarope/Comp-Sci-Homework-3-/main/English%20Traning%20Hyponyms.csv")
 
# converting column data to list
hypo_eng_train = data['Hyponyms'].tolist()
 
# printing list data
print('Hyponyms:', hypo_eng_train)


## Hearst data (makes it easier so you don't have to run it)
#Reads the csv of the Hearst data
data_hearst = read_csv('https://raw.githubusercontent.com/MattyXarope/Comp-Sci-Homework-3-/main/Hearst%20Training%20Hyponyms.csv')
 
# converting column data to list
hypo = data_hearst['Hyponyms'].tolist()
 
# printing list data
print('Hyponyms:', hypo)


Hyponyms: ['blackfly', 'Turonian', 'abhorrence', 'tropical storm', 'militarization', 'pollution', 'photomicrograph', 'swamp gum', 'song', 'wing', 'Hero', 'cumulus', 'silver', 'sand verbena', 'Carly Fiorina', 'louse', 'sodium nitrite', 'navigator', 'Erna Brodber', 'Raoul Dufy', 'gasworks', 'farming', 'Skara Brae', 'Murder One', 'Emmett Tyrrell', 'failure rate', 'Kim Il-sung University', 'chess', 'Sachs Harbour', 'Myrdal', 'floorshow', 'lion', 'Crowds and Power', 'homophone', 'availability', 'bush', 'A1 motorway', 'Deck', 'jet stream', 'Muhammad Ali Jinnah', 'habit', 'Bennington', 'sliminess', 'subject', 'planetary science', 'lobbyist', 'Helen of Troy', 'disservice', 'carbon', 'owner-occupier', 'fault', 'North Carolina', 'contribution', 'Forest Way', 'starling', 'spinal fusion', 'trust', 'self-confidence', 'beneficiary', 'backwash', 'Younger Dryas', 'harvesting', 'hosiery', 'Ph.D.', 'megabyte', 'New Zealand', 'Newdale', 'Belfast Lough', 'certified check', 'pearl', 'playwriting', 'Sajid M

# Data Analysis for the Hearst Training Set vs the Task's Training Set

In [156]:
from pandas.core.indexers import length_of_indexer
#Sees how many of the hyponyms of the training set appear in the values from our model

#Set training list
my_list_set=set(hypo_eng_train)

#Set our hyponyms from the hearst patterns to a list
my_dict_set=set(hypo)

#Finds the intersection of the two
in_common=[]
for i in my_list_set.intersection(my_dict_set):
  in_common.append(i)

#Write the in common hyponyms to a csv file, commented out to not do it over and over again
hypo_in_common = pd.DataFrame(in_common, columns=['Hyponyms in Common'])
hypo_in_common.to_csv('Hyponym_in_Common.csv')

#Data analysis for the two training sets
length_in_common = len(in_common)
length_our_training = len(hypo)
percent_in_common_ours = (round((length_in_common/length_our_training)*100))
percent_in_common_task_training = (round((length_in_common/len(my_list_set))*100))

print(f'There are {length_in_common} hyponyms in common between the training set from the task and our training set from Gemmas corpus. This represents {percent_in_common_ours}% of our training set, and {percent_in_common_task_training}% of the the training set from the task.')   

There are 90 hyponyms in common between the training set from the task and our training set from Gemmas corpus. This represents 1% of our training set, and 6% of the the training set from the task.


# Get hypernyms for list of hyponyms in common

In [200]:
## Training data from English task
#Reads the csv
task_hypos_in_common = read_csv('https://raw.githubusercontent.com/MattyXarope/Comp-Sci-Homework-3-/main/Hyponym_in_Common.csv')
 
# converting column data to list
hyponym_list_in_common = list(task_hypos_in_common['Hyponyms in Common'])

#Get hypernyms from the hyponyms in common uses function to get hypernym from input term 
list_of_hypers = []
for i in hyponym_list_in_common: 
  list_of_hypers.append(get_hps(hps_dict, i))

#Write the hypernyms to a csv file, commented out to not do it over and over again
# columns = ['Hyponyms','Hypernyms Corpus']
# data = list(zip(hyponym_list_in_common, list_of_hypers))
# df = pd.DataFrame(data=data, columns=columns)
# df.to_csv('Common Hyponyms and Hypernyms from Corpus Data.csv')

# Import Hyponyms in Common with data from Task and Corpus

In [210]:
## Training data from English task
#Reads the csv
csv_common_hyper_hypos = read_csv('https://raw.githubusercontent.com/MattyXarope/Comp-Sci-Homework-3-/main/Common%20Hyponyms%20and%20Hypernyms%20from%20Corpus%20Data.csv')

# Set columns to variables
Hyponyms = csv_common_hyper_hypos['Hyponyms']
Hypernyms_Corpus = csv_common_hyper_hypos['Hypernyms Corpus']
Hypernyms_Task = csv_common_hyper_hypos['Hypernyms Task']